In [34]:
import json
import random
# import utils

import datasets
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorForSeq2Seq


def load_from_dialogsum(file_path):
    ''' load dialoguesum jsonl data '''
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    data_dict = {}
    for sample in data:
        data_dict[sample['fname']] = {
            'summary1': sample['summary1'],
            'topic1': sample['topic1'],
            'summary2': sample['summary2'],
            'topic2': sample['topic2'],
            'summary3': sample['summary3'],
            'topic3': sample['topic3']}

    return data_dict

In [35]:
dialogsum_test = load_from_dialogsum("../data/dialogsum/dialogsum.test.jsonl")
print(dialogsum_test['test_0'])

{'summary1': 'Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.', 'topic1': 'communication method', 'summary2': 'In order to prevent employees from wasting time on Instant Message programs, #Person1# decides to terminate the use of those programs and asks Ms. Dawson to send out a memo to all employees by the afternoon.', 'topic2': 'company policy', 'summary3': 'Ms. Dawson takes a dictation for #Person1# about prohibiting the use of Instant Message programs in the office. They argue about its reasonability but #Person1# still insists.', 'topic3': 'dictation'}


In [36]:
def load_from_result(file_path):
    ''' load result json data '''
    data = []
    
    with open(file_path, 'r') as f:
        for line in f:
            data.extend(json.loads(line))
    # print(data)
    data_dict = {}
    for sample in data:
        try:
                data_dict[sample['fname']] = {
                'gen_summary1': sample['gen_summary1'],
                'gen_summary2': sample['gen_summary2'],
                'gen_summary3': sample['gen_summary3']
            }
        except:
            pass
    return data_dict

In [37]:
result = load_from_result("../result/bart-topic-length-cosine.json")
print(result['test_0'])

{'gen_summary1': '#Person1# asks Ms. Dawson to take a dictation for him and tells the staff that the use of Instant Message programs by employees during working hours is strictly prohibited. The memo applies to internal and external communications. Any employee who persists in using Instant Messaging will first receive a warning and be placed on probation.', 'gen_summary2': '#Person1# asks Ms. Dawson to take a dictation for him and tells the staff that the use of Instant Message programs by employees during working hours is strictly prohibited. The policy applies to internal and external communications. Any employee who persists in using Instant Messaging will first receive a warning and be placed on probation, and at second offense, the employee will face termination.', 'gen_summary3': '#Person1# asks Ms. Dawson to take a dictation for him and tells her that all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by empl

In [38]:
def show_all_result(test_id, num):
    topic_no = "topic" + str(num)
    summary_no = "summary" + str(num)
    dialogsum_test = load_from_dialogsum("../data/dialogsum/dialogsum.test.jsonl")
    print("Topic: ",dialogsum_test[test_id][topic_no])
    print('*'*50 + summary_no + '*'*50)
    print(dialogsum_test[test_id][summary_no])
    list_method = ['bart-topic-length-prompt-contrastive-topic-tagger']
    # list_method = ['bart-topic-length-cosine-negative-topic-tagger-decoder']
    for i in list_method:
        print('-'*50 + i + '-'*50)
        path = f"../result/{i}.json"
        result = load_from_result(path)
        print(result[test_id]['gen_' + summary_no])

In [39]:
show_all_result('test_0', 1)

Topic:  communication method
**************************************************summary1**************************************************
Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.
--------------------------------------------------bart-topic-length-prompt-contrastive-topic-tagger--------------------------------------------------
Summary: #Person1# and #Person2# are talking about the presidential election. They both voted for Trump, but they don't think he is the right person.


In [40]:
test_groundtruth = load_from_dialogsum("../data/dialogsum/dialogsum.test.jsonl")
test_predict = load_from_result("../result/bart-topic-length-prompt-contrastive-topic-tagger.json")

In [41]:
test_groundtruth['test_9']

{'summary1': '#Person1# and #Person2# are talking about Abraham Lincoln. They think he was a noble man.',
 'topic1': 'sound character',
 'summary2': '#Person2# admires Abraham Lincoln for his perseverance, courage and humility.',
 'topic2': 'famous people',
 'summary3': '#Person1# and #Person2# talk about Abraham Lincoln and his glorious history. They both admire him.',
 'topic3': 'discuss Abraham Lincoln'}

In [42]:
test_predict['test_9']

KeyError: 'test_9'

In [43]:
test_predict_list = []
test_groundtruth_list = []
for i in range(len(test_predict.keys())):
    test_predict_list.append(test_predict[f'test_{i}']['gen_summary1'])
    test_predict_list.append(test_predict[f'test_{i}']['gen_summary2'])
    test_predict_list.append(test_predict[f'test_{i}']['gen_summary3'])
    test_groundtruth_list.append(test_groundtruth[f'test_{i}']['summary1'])
    test_groundtruth_list.append(test_groundtruth[f'test_{i}']['summary2'])
    test_groundtruth_list.append(test_groundtruth[f'test_{i}']['summary3'])

KeyError: 'test_9'

In [44]:
new_test_predict = []
new_test_groundtruth = []
for sample_predict, smaple_groundtruth in zip(test_predict, test_groundtruth):
    try:
        gen_sum = sample_predict.split('Summary: ')[1]
        new_test_predict.append(gen_sum)
    except:
        new_test_predict.append(sample_predict)
        new_test_groundtruth.append(smaple_groundtruth)
    truth_sum = smaple_groundtruth.split('Summary: ')[1]
    new_test_groundtruth.append(truth_sum)

IndexError: list index out of range

In [23]:
show_all_result('test_0', 2)

Topic:  company policy
**************************************************summary2**************************************************
In order to prevent employees from wasting time on Instant Message programs, #Person1# decides to terminate the use of those programs and asks Ms. Dawson to send out a memo to all employees by the afternoon.
--------------------------------------------------bart-topic-length-cosine--------------------------------------------------
#Person1# asks Ms. Dawson to take a dictation for him and tells the staff that the use of Instant Message programs by employees during working hours is strictly prohibited. The policy applies to internal and external communications. Any employee who persists in using Instant Messaging will first receive a warning and be placed on probation, and at second offense, the employee will face termination.
--------------------------------------------------bart-topic-length-cosine-negative--------------------------------------------------

In [24]:
show_all_result('test_0', 3)

Topic:  dictation
**************************************************summary3**************************************************
Ms. Dawson takes a dictation for #Person1# about prohibiting the use of Instant Message programs in the office. They argue about its reasonability but #Person1# still insists.
--------------------------------------------------bart-topic-length-cosine--------------------------------------------------
#Person1# asks Ms. Dawson to take a dictation for him and tells her that all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited. Any employee who persists in using Instant Messaging will be placed on probation.
--------------------------------------------------bart-topic-length-cosine-negative--------------------------------------------------
#Person1# asks Ms. Dawson to take a dictation for him and tells her the use of Instant Message programs 

In [28]:
show_all_result('test_101', 1)

Topic:  use the ATM
**************************************************summary1**************************************************
#Person1# doesn't know how to use the ATM. #Person2# teaches #Person1# step by step.
--------------------------------------------------bart-topic-length-cosine--------------------------------------------------
#Person2# shows #Person1# how to use the ATM and tells #Person 1# it's pretty easy.
--------------------------------------------------bart-topic-length-cosine-negative--------------------------------------------------
#Person2# shows #Person1# how to use the ATM, which is a new experience.
--------------------------------------------------bart-topic-length-cosine-negative-topic-tagger--------------------------------------------------
#Person2# teaches #Person1# how to use the ATM and tells #Person 1# what to do.
--------------------------------------------------bart-topic-length-cosine-negative-prompt-tagger-margin---------------------------------------

In [29]:
show_all_result('test_101', 2)

Topic:  ATM
**************************************************summary2**************************************************
#Person1# doesn't know how to use an ATM. #Person2# teaches #Person1#.
--------------------------------------------------bart-topic-length-cosine--------------------------------------------------
#Person2# teaches #Person1# how to use the ATM.
--------------------------------------------------bart-topic-length-cosine-negative--------------------------------------------------
#Person2# teaches #Person1# how to use the ATM.
--------------------------------------------------bart-topic-length-cosine-negative-topic-tagger--------------------------------------------------
#Person2# teaches #Person1# how to use the ATM.
--------------------------------------------------bart-topic-length-cosine-negative-prompt-tagger-margin--------------------------------------------------
#Person2# teaches #Person1# how to use the ATM.
--------------------------------------------------bart-

In [30]:
show_all_result('test_101', 3)

Topic:  ATM using
**************************************************summary3**************************************************
#Person2# is teaching #Person1# how to use an ATM step by step because #Person1# doesn't know how to use.
--------------------------------------------------bart-topic-length-cosine--------------------------------------------------
#Person1# wants to use the ATM but doesn't know how. #Person2# shows #Person1 # how to use it.
--------------------------------------------------bart-topic-length-cosine-negative--------------------------------------------------
#Person1# needs to use the ATM but doesn't know how. #Person2# teaches #Person1 # how to use it.
--------------------------------------------------bart-topic-length-cosine-negative-topic-tagger--------------------------------------------------
#Person2# teaches #Person1# how to use the ATM and tells #Person 1# to slide the card into the machine and type the PIN in.
---------------------------------------------